In [38]:
import numpy as np
import pandas as pd
import requests
import pprint
from tqdm import tqdm #progress bar library -- not necessary, but helpful since some of this is slower

In [2]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 150)
%matplotlib inline

In [29]:
my_team_id = 5171826

In [30]:
#pull the leagues that the selected team is in
url = f'https://fantasy.premierleague.com/api/entry/{my_team_id}/'
r = requests.get(url)
json = r.json()

league_list = {}
for id_num in np.arange(len(json['leagues']['classic'])):
    if json['leagues']['classic'][id_num]['league_type'] != 's':
        league_list[json['leagues']['classic'][id_num]['id']] = json['leagues']['classic'][id_num]['name']


In [31]:
league_list

{889368: 'Beat the Overlord', 1154028: 'Go on my Heung-min Son'}

In [32]:
selected_league = 1154028

In [34]:
# pull info on the selected league
# make the league id a variable

league_url = f'https://fantasy.premierleague.com/api/leagues-classic/{selected_league}/standings/'
league_r = requests.get(league_url)
league_json = league_r.json()

In [44]:
league_df = pd.DataFrame(league_json['standings']['results'])

In [45]:
league_df

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name
0,32062212,15,Jackson Seidl,1,1,1,2097,3777451,Intelligent C&%$s
1,34294458,27,Trevor Kellogg,2,2,2,1995,5083920,Messi City F.C.
2,33017527,12,John Masla,3,3,3,1970,5187741,Masla
3,34041032,25,Charles von Reis-Iglesias,4,4,4,1964,5152592,Chunk
4,32830630,8,Chandler Foster,5,5,5,1857,5116496,BI United
5,32909403,23,Andrew Peters,6,6,6,1803,5171826,ahj_time
6,34120975,20,Willie Hayward,7,7,7,1466,5330029,Pep2020
7,34382330,19,Will Diiorio,8,8,8,1287,5359191,Wills Team


In [47]:
league_entries = list(league_df.entry)

In [48]:
league_entries

[3777451, 5083920, 5187741, 5152592, 5116496, 5171826, 5330029, 5359191]

In [164]:
current_gw = 34
entry_id = league_entries[0]
league_teams_df = pd.DataFrame()

for entry_id in league_entries:
    teams_url = f'https://fantasy.premierleague.com/api/entry/{entry_id}/event/{current_gw}/picks/'
    teams_r = requests.get(teams_url)
    teams_json = teams_r.json()

    single_team_df = pd.DataFrame(teams_json['picks'])
    single_team_df['entry'] = entry_id

    league_teams_df = league_teams_df.append(single_team_df)


In [165]:
league_teams_df = league_teams_df.merge(league_df, on = 'entry')

In [200]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()

In [201]:
player_names = pd.DataFrame(json['elements']).loc[:, ['id', 'web_name', 'team']]

In [202]:
players_url = f'https://fantasy.premierleague.com/api/event/{current_gw}/live/'
players_r = requests.get(players_url)
players_json = players_r.json()

In [203]:
live_pts_df = pd.DataFrame(players_json['elements'])

In [204]:
#unpack the stats column (in json form) to columns, and then merge back with the original dataframe
unpacked_stats_df = pd.DataFrame(live_pts_df.stats.values.tolist())
live_pts_df = live_pts_df.join(unpacked_stats_df)

In [207]:
live_pts_df = live_pts_df.merge(player_names)

In [208]:
#renaming the 'id' column to 'elements' to match other tables
live_pts_df = live_pts_df.rename(columns = {'id': 'element'})

In [210]:
live_pts_df = live_pts_df[['element', 'total_points', 'web_name']]

In [211]:
#merge in live player data with the league teams df, to see how each manager's team has done so far (at the player level)
league_teams_df = league_teams_df.merge(live_pts_df).sort_values('rank')

In [212]:
league_df.loc[:, ['player_name', 'entry']]

,player_name,entry
0,Jackson Seidl,3777451
1,Trevor Kellogg,5083920
2,John Masla,5187741
3,Charles von Reis-Iglesias,5152592
4,Chandler Foster,5116496
5,Andrew Peters,5171826
6,Willie Hayward,5330029
7,Will Diiorio,5359191


In [213]:
#pick a team to compare to 
comparison_team = 5116496

In [214]:
my_players = list(league_teams_df[league_teams_df.entry == my_team_id]['element'])

In [215]:
comparison_players = list(league_teams_df[league_teams_df.entry == comparison_team]['element'])

In [216]:
def find_distinct_players(team1, team2):
    return (list(list(set(team1)-set(team2)) + list(set(team2)-set(team1))))

In [217]:
distinct_players = find_distinct_players(my_players, comparison_players)

In [220]:
#show differential players (note that this view doesn't show players on bench)

diffs_df = league_teams_df[(league_teams_df.entry.isin([my_team_id, comparison_team])) & (league_teams_df.element.isin(distinct_players))
    & (league_teams_df.multiplier != 0)]

In [236]:
for entry in diffs_df.entry.unique():
    section = diffs_df[diffs_df.entry == entry]
    pprint.pprint(section.entry_name.values[0])
    pprint.pprint(section.loc[:, ['web_name', 'total_points', 'is_captain']])
    print('\n')

'BI United'
     web_name  total_points  is_captain
22       Kane             0        True
38    Pereira             0       False
6        Jota             0       False
89     Dallas             1       False
56  Fernandes             0       False


'ahj_time'
    web_name  total_points  is_captain
94   Bamford             1       False
97   Pereira             0       False
98  Harrison             2       False
66     Salah             0       False
82   Meslier             2       False


